# Import Libraries

In [274]:
import pandas as pd 
import numpy as np
import logging
from datetime import datetime
import requests
from io import StringIO
from scipy.stats import beta
import math

# Global Configuration

In [275]:
# Back-test date range
BACKTEST_START     = '2021-01-01' 
BACKTEST_END       = '2025-11-07' 
# BACKTEST_START     = '2011-06-01' 
# BACKTEST_END       = '2025-06-01' 

# Rolling window length (in months)
INVESTMENT_WINDOW  = 12

# Step frequency for window start-dates: 'Daily', 'Weekly' or 'Monthly'
PURCHASE_FREQ      = 'Daily'

# Minimum per-period weight (to avoid zero allocations)
MIN_WEIGHT         = 1e-5

PURCHASE_FREQ_TO_OFFSET = {
    'Daily':   '1D',
    'Weekly':  '7D',
    'Monthly': '1M',
}

In [276]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Download BTC Data

In [277]:
try:
    from coinmetrics.api_client import CoinMetricsClient
except ImportError:
    raise ImportError("coinmetrics.api_client module is required. Install it via pip:\n\n    pip install coinmetrics-api-client")

def extract_btc_data_to_csv(local_path='btc_data.csv'):
    # Coin Metrics BTC CSV (raw GitHub URL)
    url = "https://raw.githubusercontent.com/coinmetrics/data/master/csv/btc.csv"
    
    # Download the content
    response = requests.get(url)
    response.raise_for_status()  # raises an error for bad responses
    
    # Parse CSV content
    btc_df = pd.read_csv(StringIO(response.text))

    btc_df['time'] = pd.to_datetime(btc_df['time']).dt.normalize()
    btc_df['time'] = btc_df['time'].dt.tz_localize(None)
    btc_df.set_index('time', inplace=True)

    btc_df.to_csv(local_path)
    
    # Show the df
    btc_df

btc_df = extract_btc_data_to_csv("btc_data.csv")

# Load Data

In [278]:
def load_data():
    df = pd.read_csv("btc_data.csv", index_col=0, parse_dates=True)
    df = df.loc[~df.index.duplicated(keep='last')]
    df = df.sort_index()
    return df

def validate_price_data(df):
    if df.empty or 'PriceUSD' not in df.columns:
        raise ValueError("Invalid BTC price data.")
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("Index must be datetime.")

# Strategy

## Hyper Parameters

In [279]:
_FULL_FEATURES = None

MIN_W = 1e-5
WINS = [30, 90, 180, 365, 1461]
FEATS = [f"z{w}" for w in WINS]
PROTOS = [(0.5, 5.0), (1.0, 1.0), (5.0, 0.5)]

# Optimized theta parameters from the final model run
THETA = np.array([1.6741, 0.0805, 0.4075, -0.0023,
 1.2621, 2.1404, -0.7984, -0.0140,
 -0.1359, 0.0563, -0.5971, -1.0053,
 -0.8718, -0.0651, -0.2813, -0.0571,
 -0.6538, -1.1288, 0.0000, 0.0000,
 0.3622, 1.9998, 5.0000])

## Helper Functions

In [280]:
def softmax(x: np.ndarray) -> np.ndarray:
    """Converts a vector of scores into a probability distribution."""
    ex = np.exp(x - x.max())
    return ex / ex.sum()

def allocate_sequential(raw: np.ndarray) -> np.ndarray:
    """Strict left-to-right 'drain' allocator."""
    n = len(raw)
    floor = n * MIN_W
    rem_budget, rem_raw = 1 - floor, raw.sum()
    w = np.empty_like(raw)
    for i, x in enumerate(raw):
        share = 0 if rem_raw == 0 else (x / rem_raw) * rem_budget
        w[i] = MIN_W + share
        rem_budget -= share
        rem_raw -= x
    return w / w.sum()

def beta_mix_pdf(n: int, mix: np.ndarray) -> np.ndarray:
    """Generates a smooth baseline curve from a mixture of Beta distributions."""
    t = np.linspace(0.5 / n, 1 - 0.5 / n, n)
    return (mix[0] * beta.pdf(t, *PROTOS[0]) +
            mix[1] * beta.pdf(t, *PROTOS[1]) +
            mix[2] * beta.pdf(t, *PROTOS[2])) / n

def zscore(s: pd.Series, win: int) -> pd.Series:
    """Calculates the rolling z-score for a given series and window."""
    m = s.rolling(win, win // 2).mean()
    sd = s.rolling(win, win // 2).std()
    return ((s - m) / sd).fillna(0)

## Main Strategy Functions

In [ ]:
MSTR_TAU_DAYS: float = 80     # Time-decay τ
MSTR_CAP_MAX: float  = 1.2000        # Boost accumulation upper cap
BOOST_SMALL: float   = 1.0500        # Jenks group 1 boost value
BOOST_MED: float     = 1.1000        # Jenks group 2 boost value
BOOST_LARGE: float   = 1.1500        # Jenks group 3 boost value

ZCLIP: float         = 3.0000        # Compatibility suppression threshold z_clip
SOFTMAX_T: float     = 2.0000        # Softmax temperature for the 30/90/180-day windows

MSTR_EVENTS_CSV: str = "mstr_buy_signal.csv"  # Path to the historical MSTR purchase event file


def _jenks_or_quantile_breaks(btc_amount: pd.Series, nb_class: int = 4) -> list:
    """
    Prefer Jenks natural breaks; if the jenkspy package is unavailable, fall back to quantile approximation.
    Returns a list of breaks (length nb_class+1, sorted), e.g., [min, cut1, cut2, max].
    """
    vals = btc_amount.dropna().astype(float).values
    vals.sort()
    if vals.size == 0:
        # No data available — return a fallback range
        return [0.0, 500.0, 2000.0, 1e9]

    # Try Jenks
    try:
        import jenkspy
        brks = jenkspy.jenks_breaks(vals, nb_class=nb_class)
        # Ensure monotonicity and valid boundaries
        brks[0] = float(vals[0])
        brks[-1] = float(vals[-1])
        return brks
    except Exception:
        # Fallback: approximate using quantiles (33% / 67%)
        q1 = float(np.quantile(vals, 1/3))
        q2 = float(np.quantile(vals, 2/3))
        return [float(vals[0]), q1, q2, float(vals[-1])]


def _tier_boost_from_amount(x: float, breaks: list) -> float:
    """
    Map a single btc_amount value to the boost levels 1.05 / 1.10 / 1.15
    based on the Jenks/quantile breakpoints.
    breaks = [b0, b1, b2, b3]; the intervals are:
      < b1 → small;  b1~b2 → medium;  ≥ b2 → large
    """
    _, b1, b2, _ = breaks
    if x < b1:
        return BOOST_SMALL
    elif x < b2:
        return BOOST_MED
    else:
        return BOOST_LARGE


def _load_mstr_events(csv_path: str = MSTR_EVENTS_CSV) -> pd.DataFrame:
    """
    Load MSTR purchase records and return a DataFrame with the following columns:
      - date: pd.Timestamp
      - btc_amount: float
      - boost: float (determined by tier)
    """
    df = pd.read_csv(csv_path)
    # Expected column names: purchase_date, btc_amount, usd_amount
    if "purchase_date" not in df or "btc_amount" not in df:
        raise ValueError("MSTR CSV needs 'purchase_date' and 'btc_amount' column")
    df["date"] = pd.to_datetime(df["purchase_date"])
    df = df.sort_values("date").reset_index(drop=True)

    breaks = _jenks_or_quantile_breaks(df["btc_amount"], nb_class=3)
    df["boost"] = df["btc_amount"].astype(float).apply(lambda x: _tier_boost_from_amount(x, breaks))
    return df[["date", "btc_amount", "boost"]]


def _mstr_decay_sum(index: pd.DatetimeIndex, events_df: pd.DataFrame) -> np.ndarray:
    """
    For each day d, sum all (boost - 1) * exp(-(d - t_i) / τ) contributions from past events
    to obtain f_MSTR(d) (with cap applied).
    Returns a vector of shape (len(index),).
    """
    if events_df is None or events_df.empty:
        return np.ones(len(index), dtype=float)

    ev_dates = events_df["date"].dt.tz_localize(None)
    ev_boost = events_df["boost"].astype(float).values

    # Convert timestamps to ordinal days (or use numpy datetime64) for faster day-difference computation
    ev_ord = ev_dates.view("int64") // (24*3600*1_000_000_000)  # 天數
    idx_ord = pd.Series(index).dt.tz_localize(None).view("int64") // (24*3600*1_000_000_000)

    out = np.ones(len(index), dtype=float)
    for j, d_ord in enumerate(idx_ord):
        # Select all t_i <= d
        diffs = d_ord - ev_ord
        mask = diffs >= 0
        if not mask.any():
            out[j] = 1.0000
            continue

        # Exponential decay: exp(-Δ / τ)
        decay = np.exp(-(diffs[mask].astype(float) / MSTR_TAU_DAYS))
        increments = (ev_boost[mask] - 1.0000) * decay
        f = 1.0000 + increments.sum()

        # Apply cap
        out[j] = min(f, MSTR_CAP_MAX)

    return out


def _z_suppress_value(feat_slice_row: pd.Series) -> float:
    """
    Compute the suppression variable z_sup using the smooth maximum (softmax / LogSumExp)
    of the 30/90/180-day z-scores.
    Returns a non-negative z_sup.
    """
    z30 = float(feat_slice_row.get("z30", 0.0))
    z90 = float(feat_slice_row.get("z90", 0.0))
    z180 = float(feat_slice_row.get("z180", 0.0))
    # Focus only on the overheating side; softmax over positive values is effectively equivalent here
    # (z <= 0 does not trigger suppression)
    nums = np.array([z30, z90, z180], dtype=float)
    # LogSumExp / softmax smooth maximum
    s = np.exp(nums / SOFTMAX_T).mean()
    z_sup = max(0.0, SOFTMAX_T * math.log(s))
    return z_sup


def _compatibility_suppression(z_sup: float, z_clip: float = ZCLIP) -> float:
    """
    Linear suppression: supp = max(0, 1 - z_sup / z_clip)
    """
    # return max(0.0000, 1.0000 - (z_sup / z_clip))
    return z_sup


def compute_mstr_multiplier(feat_slice: pd.DataFrame, mstr_csv_path: str = MSTR_EVENTS_CSV) -> np.ndarray:
    """
    For the dates in df_window (feat_slice.index), return a vector f_MSTR_eff(d) with the same length.
    Steps:
      1) Load events → Jenks classification → convert to boost values
      2) Apply time-decay accumulation (with cap = 1.20)
      3) Apply price compatibility suppression (smooth maximum over 30/90/180; z_clip = 3)
    """
    try:
        events_df = _load_mstr_events(mstr_csv_path)
    except Exception:
        events_df = pd.DataFrame(columns=["date", "btc_amount", "boost"])

    # 2) Time decay, accumulation, and cap
    f_mstr = _mstr_decay_sum(feat_slice.index, events_df)  # shape = (n_days,)

    # 3) Price compatibility suppression (day-by-day)
    f_eff = np.empty_like(f_mstr)
    for i, (_, row) in enumerate(feat_slice.iterrows()):
        z_sup = _z_suppress_value(row)                # 30/90/180 softmax
        supp  = _compatibility_suppression(z_sup)     # Linear suppression, z_clip = 3
        f_eff[i] = 1.0000 + (f_mstr[i] - 1.0000) * supp

    return f_eff

In [ ]:
def construct_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates features on the full historical data ONCE and returns the
    relevant slice. This robustly handles calls from different boilerplate
    functions and avoids boundary errors that cause leakage.
    """
    global _FULL_FEATURES

    # Only compute the full feature set if it hasn't been done yet.
    if _FULL_FEATURES is None:
        try:
            # Assumes 'btc_data.csv' is in the same directory.
            full_price_df = pd.read_csv("btc_data.csv", index_col=0, parse_dates=True)
        except FileNotFoundError:
            raise FileNotFoundError("btc_data.csv not found. Please ensure it's in the correct directory.")
        
        # Select only the PriceUSD column before doing anything else.
        full_price_df = full_price_df[['PriceUSD']]
        
        # We need history from before the backtest start date for rolling windows.
        full_price_df = full_price_df.loc["2020-02-18":]
        # full_price_df = full_price_df.loc["2010-07-18":] 

        log_prices = np.log(full_price_df['PriceUSD'])
        
        z_all = pd.DataFrame({f"z{w}": zscore(log_prices, w).clip(-4, 4) for w in WINS}, index=log_prices.index)
        
        # The strategy uses lagged features to avoid look-ahead bias.
        z_lag = z_all.shift(1).fillna(0)
        
        _FULL_FEATURES = full_price_df.join(z_lag)

    # Return the portion of the pre-computed features that matches the input index.
    return _FULL_FEATURES.reindex(df.index).fillna(0)


def compute_weights(df_window: pd.DataFrame) -> pd.Series:
    """
    Given a slice of data, computes portfolio weights that sum to 1.
    This function first calls construct_features to ensure the necessary
    feature columns are present.
    """
    if df_window.empty:
        return pd.Series(dtype=float)

    feat_slice = construct_features(df_window)

    alpha, beta_v = THETA[:18].reshape(3, 6), THETA[18:]
    
    # Use features from the first day to set the annual strategy
    first_day_feats = feat_slice[FEATS].iloc[0].values
    mix = softmax(alpha @ np.r_[1, first_day_feats])
    
    # Calculate the components of the allocation formula
    n_days = len(feat_slice)
    base_alloc = beta_mix_pdf(n_days, mix)
    dynamic_signal = np.exp(-(feat_slice[FEATS].values @ beta_v))

    f_mstr_eff = compute_mstr_multiplier(feat_slice, mstr_csv_path=MSTR_EVENTS_CSV)

    # Combine signals and compute final weights
    raw_weights = base_alloc * dynamic_signal * f_mstr_eff
    final_weights = allocate_sequential(raw_weights)
    
    return pd.Series(final_weights, index=feat_slice.index)

# Run Strategy

In [283]:
def _make_window_label(window_start: pd.Timestamp, window_end: pd.Timestamp) -> str:
    """
    Format "YYYY-MM-DD → YYYY-MM-DD" for a rolling window.
    """
    start_str = pd.to_datetime(window_start).strftime("%Y-%m-%d")
    end_str   = pd.to_datetime(window_end).strftime("%Y-%m-%d")
    return f"{start_str} → {end_str}"

In [284]:
def compute_cycle_spd(
    dataframe: pd.DataFrame,
    strategy_function
) -> pd.DataFrame:
    """
    Compute sats‐per‐dollar (SPD) stats over rolling windows.

    - Uses full‐history features (no look‐ahead).
    - Window length = INVESTMENT_WINDOW months.
    - Step every PURCHASE_FREQ.
    - Returns a DataFrame indexed by window label, with:
        min_sats_per_dollar, max_sats_per_dollar,
        uniform_sats_per_dollar, dynamic_sats_per_dollar,
        uniform_percentile, dynamic_percentile, excess_percentile.
    """
    # 1) Precompute full-history features & restrict to backtest
    full_feat = construct_features(dataframe).loc[BACKTEST_START:BACKTEST_END]

    # 2) Window parameters
    window_offset  = pd.DateOffset(months=INVESTMENT_WINDOW)
    step_freq      = PURCHASE_FREQ_TO_OFFSET[PURCHASE_FREQ]

    results = []
    for window_start in pd.date_range(
        start=pd.to_datetime(BACKTEST_START),
        end=pd.to_datetime(BACKTEST_END) - window_offset,
        freq=step_freq
    ):
        window_end  = window_start + window_offset
        feat_slice  = full_feat.loc[window_start:window_end]
        price_slice = dataframe["PriceUSD"].loc[window_start:window_end]

        if price_slice.empty:
            continue

        label       = _make_window_label(window_start, window_end)
        inv_price   = (1.0 / price_slice) * 1e8  # sats per dollar

        # Compute weights on this slice
        weight_slice = strategy_function(feat_slice)

        # Uniform vs. dynamic SPD
        uniform_spd = inv_price.mean()
        dynamic_spd = (weight_slice * inv_price).sum()

        # Min/max for percentile scaling
        min_spd = inv_price.min()   # low price → high SPD
        max_spd = inv_price.max()   # high price → low SPD
        span    = max_spd - min_spd

        uniform_pct = (uniform_spd - min_spd) / span * 100
        dynamic_pct = (dynamic_spd - min_spd) / span * 100

        results.append({
            "window":                   label,
            "min_sats_per_dollar":      min_spd,
            "max_sats_per_dollar":      max_spd,
            "uniform_sats_per_dollar":  uniform_spd,
            "dynamic_sats_per_dollar":  dynamic_spd,
            "uniform_percentile":       uniform_pct,
            "dynamic_percentile":       dynamic_pct,
            "excess_percentile":        dynamic_pct - uniform_pct,
        })

    return pd.DataFrame(results).set_index("window")

# Backtest

In [285]:
def backtest_dynamic_dca(
    dataframe: pd.DataFrame,
    strategy_function,
    *,
    strategy_label: str = "strategy"
) -> pd.DataFrame:
    """
    1) Runs compute_cycle_spd(...)
    2) Prints aggregated min/max/mean/median of dynamic SPD
    3) Prints aggregated SPD percentiles
    4) Computes & prints exponentially-decayed average SPD and percentile
    5) Returns the full SPD table.

    Exponential decay:
      • decay_rate ∈ (0,1): lower → faster decay
      • most recent window has highest weight
      • weights normalized to sum to 1
    """
    # --- run the rolling-window SPD backtest
    spd_table   = compute_cycle_spd(dataframe, strategy_function)
    dynamic_spd = spd_table["dynamic_sats_per_dollar"]
    dynamic_pct = spd_table["dynamic_percentile"]

    # --- print standard aggregated metrics
    print(f"\nAggregated Metrics for {strategy_label}:")
    print("Dynamic Sats-per-Dollar:")
    for stat in ("min", "max", "mean", "median"):
        val = getattr(dynamic_spd, stat)()
        print(f"  {stat}: {val:.2f}")

    print("\nDynamic SPD Percentiles:")
    for stat in ("min", "max", "mean", "median"):
        val = getattr(dynamic_pct, stat)()
        print(f"  {stat}: {val:.2f}%")

    # --- exponential decay weighting
    decay_rate = 0.9
    N = len(dynamic_spd)
    # weight for window i (0 = oldest, N-1 = newest)
    raw_weights = np.array([decay_rate ** (N - 1 - i) for i in range(N)])
    exp_weights = raw_weights / raw_weights.sum()

    # --- compute decayed averages
    exp_avg_spd = (dynamic_spd.values * exp_weights).sum()
    exp_avg_pct = (dynamic_pct.values * exp_weights).sum()

    # --- print decayed metrics
    print(f"\nExponential-Decay Average SPD: {exp_avg_spd:.2f}")
    print(f"Exponential-Decay Average SPD Percentile: {exp_avg_pct:.2f}%")

    return spd_table

In [286]:
def check_strategy_submission_ready(
    dataframe: pd.DataFrame,
    strategy_function
) -> None:
    """
    Sanity-check that `strategy_function`:
      1. Uses no future data (forward-leakage test).
      2. Produces weights ≥ MIN_WEIGHT.
      3. Sums to 1.0 in each rolling window.
      4. Outperforms uniform DCA in at least 50% of rolling windows.
    """
    passed = True

    # ──────────────────────────────────────────────────────────
    # 1) Forward-leakage test
    # ──────────────────────────────────────────────────────────
    backtest_df  = dataframe.loc[BACKTEST_START:BACKTEST_END]
    full_weights = strategy_function(dataframe) \
                       .reindex(backtest_df.index) \
                       .fillna(0.0)

    step_dates = max(len(backtest_df) // 50, 1)
    probe_dates = backtest_df.index[::step_dates]

    for probe in probe_dates:
        masked = dataframe.copy()
        masked.loc[masked.index > probe, :] = np.nan

        masked_wt = strategy_function(masked) \
                        .reindex(full_weights.index) \
                        .fillna(0.0) 

        if not np.isclose(masked_wt.loc[probe],
                          full_weights.loc[probe],
                          rtol=1e-9, atol=1e-12):
            delta = abs(masked_wt.loc[probe] - full_weights.loc[probe])
            print(f"[{probe.date()}] ❌ Forward-leakage detected (Δ={delta:.2e})")
            passed = False
            break

    # ──────────────────────────────────────────────────────────
    # 2) Weight checks per rolling window
    # ──────────────────────────────────────────────────────────
    window_offset = pd.DateOffset(months=INVESTMENT_WINDOW)
    step_freq     = PURCHASE_FREQ_TO_OFFSET[PURCHASE_FREQ]

    for window_start in pd.date_range(
        start=pd.to_datetime(BACKTEST_START),
        end=pd.to_datetime(BACKTEST_END) - window_offset,
        freq=step_freq
    ):
        window_end = window_start + window_offset
        label      = _make_window_label(window_start, window_end)

        w_slice = strategy_function(dataframe.loc[window_start:window_end])

        if (w_slice <= 0).any():
            print(f"[{label}] ❌ Non-positive weights detected.")
            passed = False

        if (w_slice < MIN_WEIGHT).any():
            print(f"[{label}] ❌ Weight below MIN_WEIGHT = {MIN_WEIGHT}.")
            passed = False

        total = w_slice.sum()
        if not np.isclose(total, 1.0, rtol=1e-5, atol=1e-8):
            print(f"[{label}] ❌ Sum-to-1 check failed: {total:.4f}")
            passed = False

    # ──────────────────────────────────────────────────────────
    # 3) Performance vs. Uniform DCA (RELAXED)
    # ──────────────────────────────────────────────────────────
    spd_table = compute_cycle_spd(dataframe, strategy_function)

    underperf_records = []
    for label, row in spd_table.iterrows():
        dp, up = row["dynamic_percentile"], row["uniform_percentile"]
        if dp < up:
            underperf_records.append({
                "Window": label,
                "Dynamic Percentile": dp,
                "Uniform Percentile": up,
                "Delta": dp - up
            })

    total = len(spd_table)
    failed = len(underperf_records)
    pass_ratio = (total - failed) / total

    if underperf_records:
        df_underperf = pd.DataFrame(underperf_records)
        print("\n⚠️ Windows where strategy underperformed Uniform DCA:")
        display(df_underperf)

    print(f"\nSummary: Your strategy underperformed uniform DCA in {failed} out of {total} windows "
          f"({100 * pass_ratio:.2f}% win rate)")

    if pass_ratio >= 0.5:
        print("✅ Strategy meets performance requirement (≥ 50% win rate vs. uniform DCA).")
    else:
        print("❌ Strategy failed performance requirement (< 50% win rate vs. uniform DCA).")
        passed = False

    # ──────────────────────────────────────────────────────────
    # Final verdict
    # ──────────────────────────────────────────────────────────
    if passed:
        print("\n✅ Strategy is ready for submission.")
    else:
        print("\n⚠️ Please address the above issues before submitting.")

# Run main workflow

In [287]:
btc_df = load_data()
validate_price_data(btc_df)
btc_df = btc_df.loc[BACKTEST_START:BACKTEST_END]

In [288]:
# Rolling-window SPD backtest:
df_spd = backtest_dynamic_dca(
    btc_df,
    compute_weights,
    strategy_label="Dynamic DCA"
)

/var/folders/dr/_j6dcxqs3zn8h1179k47j1s80000gn/T/ipykernel_63912/1174490942.py:85: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  ev_ord = ev_dates.view("int64") // (24*3600*1_000_000_000)  # 天數
/var/folders/dr/_j6dcxqs3zn8h1179k47j1s80000gn/T/ipykernel_63912/1174490942.py:86: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  idx_ord = pd.Series(index).dt.tz_localize(None).view("int64") // (24*3600*1_000_000_000)
/var/folders/dr/_j6dcxqs3zn8h1179k47j1s80000gn/T/ipykernel_63912/1174490942.py:85: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  ev_ord = ev_dates.view("int64") // (24*3600*1_000_000_000)  # 天數
/var/folders/dr/_j6dcxqs3zn8h1179k47j1s80000gn/T/ipykernel_63912/1174490942.py:86: FutureWarning: Series.view is depre


Aggregated Metrics for Dynamic DCA:
Dynamic Sats-per-Dollar:
  min: 1208.40
  max: 5984.31
  mean: 3759.66
  median: 3765.25

Dynamic SPD Percentiles:
  min: 54.16%
  max: 97.21%
  mean: 81.38%
  median: 84.85%

Exponential-Decay Average SPD: 1350.72
Exponential-Decay Average SPD Percentile: 82.90%


/var/folders/dr/_j6dcxqs3zn8h1179k47j1s80000gn/T/ipykernel_63912/1174490942.py:85: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  ev_ord = ev_dates.view("int64") // (24*3600*1_000_000_000)  # 天數
/var/folders/dr/_j6dcxqs3zn8h1179k47j1s80000gn/T/ipykernel_63912/1174490942.py:86: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  idx_ord = pd.Series(index).dt.tz_localize(None).view("int64") // (24*3600*1_000_000_000)
/var/folders/dr/_j6dcxqs3zn8h1179k47j1s80000gn/T/ipykernel_63912/1174490942.py:85: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  ev_ord = ev_dates.view("int64") // (24*3600*1_000_000_000)  # 天數
/var/folders/dr/_j6dcxqs3zn8h1179k47j1s80000gn/T/ipykernel_63912/1174490942.py:86: FutureWarning: Series.view is depre

In [289]:
# Sanity checks (each window inside):
check_strategy_submission_ready(btc_df, compute_weights)

/var/folders/dr/_j6dcxqs3zn8h1179k47j1s80000gn/T/ipykernel_63912/1174490942.py:85: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  ev_ord = ev_dates.view("int64") // (24*3600*1_000_000_000)  # 天數
/var/folders/dr/_j6dcxqs3zn8h1179k47j1s80000gn/T/ipykernel_63912/1174490942.py:86: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  idx_ord = pd.Series(index).dt.tz_localize(None).view("int64") // (24*3600*1_000_000_000)
/var/folders/dr/_j6dcxqs3zn8h1179k47j1s80000gn/T/ipykernel_63912/1174490942.py:85: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  ev_ord = ev_dates.view("int64") // (24*3600*1_000_000_000)  # 天數
/var/folders/dr/_j6dcxqs3zn8h1179k47j1s80000gn/T/ipykernel_63912/1174490942.py:86: FutureWarning: Series.view is depre


Summary: Your strategy underperformed uniform DCA in 0 out of 1407 windows (100.00% win rate)
✅ Strategy meets performance requirement (≥ 50% win rate vs. uniform DCA).

✅ Strategy is ready for submission.


In [291]:
win_rate = 100
exp_decay_percentile = 82.90

score = 0.5 * win_rate + 0.5 * exp_decay_percentile
print(f"Final Model Score (50/50 weighting): {score:.2f}%")

Final Model Score (50/50 weighting): 91.45%
